In [1]:
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [2]:
# Importing data
dn_input = np.load('/home/beto/Documents/projects/DoggoNet/DN-dataset.npz')

x_train = dn_input['X_train']
x_test = dn_input['X_test']
y_train = dn_input['y_train']
y_test = dn_input['y_test']

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

# Selecting an example
# etiqueta = dn_input['y_train'][1200]
# print(etiqueta[0])
# foto = dn_input['X_train'][1200]
# foto = np.multiply(foto, 255).astype(np.int8)
# Image.fromarray(foto, 'RGB').show()

# convert class
num_classes = 4
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model initialization
model = keras.Sequential([keras.Input(shape=(240, 320, 3))])


# Defining building blocks
def add_conv_lyr(num_neurons, k_size, act_func, times=1):
    for i in range(times):
        model.add(layers.Conv2D(num_neurons, kernel_size=k_size, activation=act_func))


def add_max_pool_lyr(p_size, times=1):
    for i in range(times):
        model.add(layers.MaxPooling2D(pool_size=p_size))


def add_dense_lyr(num_neurons, act_func, times=1):
    for i in range(times):
        model.add(layers.Dense(num_neurons, activation=act_func))


# Adding logic to the model
add_conv_lyr(64, (3, 3), 'relu', 2)
add_max_pool_lyr((2, 2))
add_conv_lyr(64, (3, 3), 'relu', 2)
add_max_pool_lyr((2, 2))

add_dense_lyr(100, 'tanh', 3)

# Ending model
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
add_dense_lyr(num_classes, 'softmax')
model.summary()

batch_size = 100
epochs = 12

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print('\n-------------------- MODEL COMPILED AND READY --------------------\n\n')

Training set shape: (3400, 225, 300, 3)
Testing set shape: (600, 225, 300, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 223, 298, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 73, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 252288)            0         
_________________________________________________________________
dropout (Dropout)            (None, 252288)            0         
____________________________________________

In [3]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/12
153/153 [==============================] - 90s 589ms/step - loss: 0.7092 - accuracy: 0.7487 - val_loss: 0.2213 - val_accuracy: 0.9176
Epoch 2/12
153/153 [==============================] - 91s 597ms/step - loss: 0.1314 - accuracy: 0.9611 - val_loss: 0.1145 - val_accuracy: 0.9618
Epoch 3/12
153/153 [==============================] - 93s 608ms/step - loss: 0.0499 - accuracy: 0.9863 - val_loss: 0.1059 - val_accuracy: 0.9618
Epoch 4/12
153/153 [==============================] - 92s 602ms/step - loss: 0.0241 - accuracy: 0.9935 - val_loss: 0.2413 - val_accuracy: 0.9500
Epoch 5/12
153/153 [==============================] - 93s 605ms/step - loss: 0.0153 - accuracy: 0.9958 - val_loss: 0.0935 - val_accuracy: 0.9735
Epoch 6/12
153/153 [==============================] - 92s 602ms/step - loss: 0.0070 - accuracy: 0.9984 - val_loss: 0.0781 - val_accuracy: 0.9706
Epoch 7/12
153/153 [==============================] - 92s 601ms/step - loss: 0.1423 - accuracy: 0.9526 - val_loss: 0.3458 - val_ac

In [4]:
keras.backend.clear_session()